## Kernel Ridge Regression

In [1]:
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
import random

## Linear Kernel

Calculates the dot product of all pairwise points ($\phi_i$) in dataset D and returns that matrix

$$K_{i,j} = K(\phi_i, \phi_j) $$


$$K(\phi_i, \phi_j) = \phi_{i}^T \cdot \phi_j$$

In [2]:
def linearKernel(D):
    n = len(D)
    K = np.zeros((n, n))
    
    for i in range (n):
        for j in range (n):
            x = D[i,:]
            y = D[j,:]
            K[i][j] = np.dot(x,y)
            
    return K

## "Point " Kernels

Creates an (n x 1)-kernel matrix for a single test point, z, given a dataset D such that...

$$K_i = K(z, \phi_i)\text{         ; where $\phi_i$ $\in$ D}$$



In [3]:
def linearKernelpoint(D,z):
    n = len(D)
    K = np.zeros(n)
    
    for i in range(n):
        x = D[i,:]
        K[i] = np.dot(x, z)
        
    return np.array([K]).T


## Inhomogenous Quadratic Kernel

$$K_{i,j} = K(\phi_i, \phi_j) $$


$$ K(\phi_i, \phi_j)  = (1 + \phi_{i}^T \cdot \phi_j)^2$$

In [4]:
def quadKernel(D):
    n = len(D)
    K = np.zeros((n, n))
    
    for i in range (n):
        for j in range (n):
            x = D[i,:]
            y = D[j,:]
            K[i][j] = (1 + np.dot(x,y))**2
            
    return K

In [5]:
def quadKernelpoint(D,z):
    n = len(D)
    K = np.zeros(n)
    
    for i in range(n):
        x = D[i,:]
        K[i] = (1 + np.dot(x,z))**2
        
    return np.array([K]).T


## Gaussian Kernel

$$K_{i,j} = K(\phi_i, \phi_j) $$


$$ K(\phi_i, \phi_j)  =  exp(- \frac{\phi_{i}^T \cdot \phi_j}{2\sigma^2})$$

In [6]:
def gaussianKernel(D, sigma):
    n = len(D)
    K = np.zeros((n, n))
    
    for i in range (n):
        
        for j in range (n):
            
            x = D[i,:]
            y = D[j,:]
            diff = x - y
            K[i][j] = np.exp(- np.dot(diff,diff) / (2*sigma**2))
            
    return K

In [7]:
def gaussianKernelpoint(D,sigma, z):
    n = len(D)
    K = np.zeros(n)
    
    for i in range(n):
        
        x = D[i,:]
        diff = x - z
        K[i] = np.exp(-np.dot(diff,diff) / (2*sigma**2))
        
    return np.array([K]).T

## Kernel Ridge Regression Algorithm

![](kernel_algo.png)


In [8]:
def kernelRegression(K, Y, alpha):
    n = len(K)
    K_aug = K + 1
    K_alpha = K_aug + np.eye(n) * alpha
    K_alpha_inv = np.linalg.inv(K_alpha)
    c = np.matmul(K_alpha_inv, Y)
    Y_hat = np.matmul(K_aug, c)
    Y_hat = Y_hat[:, 0]
    
    for i in range (n):
        if (Y_hat[i] > .5):
            Y_hat[i] = 1
        else:
            Y_hat[i] = 0
    return c, np.array([Y_hat]).T

In [9]:
def testing(Z, D, c, process, sigma):
    n = len(Z)
    y_hat = np.zeros(n)
    for i in range (n):
        z = Z[i,:]
        if(process == "Linear"):
            K = linearKernelpoint(D, z)
        elif(process == "Quadratic"):
            K = quadKernelpoint(D, z)
        else:
            K = gaussianKernelpoint(D, sigma, z) 
        
        K_aug = K + 1
        val = np.matmul(c.T, K_aug)[0][0]
        if(val > .5):
            y_hat[i] = 1
        else:
            y_hat[i] = 0
            
    return np.array([y_hat]).T 

## Accuracy

$$acc = \frac{\text{number of cases where,  } \hat{y}_i = y_i}{n}$$

In [10]:
def accuracy(y, y_hat):
    y = y[:,0]
    y_hat = y_hat[:,0]
    count = 0
    for i in range(len(y)):
        if(y[i] == y_hat[i]):
            count = count + 1
    
    return count/len(y)

# Training Set

In [40]:
train = np.genfromtxt("Concrete_Data_RNorm_Class_train.txt", delimiter=',')

print("Type which kernel desired: Linear, Quadratic, Gaussian")
process = input()
print()

alpha = .001 
d = len(train[0])
D_train = train[:,np.arange(d-1)]
y_train = np.array([train[:,d-1]]).T
K = 0

if(process == "Linear"):
    K = linearKernel(D_train)
elif(process == "Quadratic"):
    K = quadKernel(D_train)
else:
    print("Choose a sigma value (must be non-zero)")
    sigma = float(input())
    print()
    K = gaussianKernel(D_train, sigma) 

c, y_hat = kernelRegression(K, y_train, alpha)
acc = accuracy(y_train, y_hat)
print("Training Accuracy: ", acc)

Type which kernel desired: Linear, Quadratic, Gaussian
Gaussian

Choose a sigma value (must be non-zero)
1.3

Training Accuracy:  0.9259708737864077


# Testing Set

In [41]:
test = np.genfromtxt("Concrete_Data_RNorm_Class_test.txt", delimiter=',')    
d = len(test[0])
D_test = test[:,np.arange(d-1)]
y_test = np.array([test[:,d-1]]).T
y_hat_test = testing(D_test, D_train, c, process, sigma)
acc_test = accuracy(y_test, y_hat_test)
print("Testing Accuracy: ",acc_test)


Testing Accuracy:  0.9611650485436893


Out of my testing $\sigma$ = 1.3 which had a testing accurary of .96